In [1]:
#import modules
from lxml import html
import re
import requests
import pandas as pd
from datetime import datetime
import os

In [2]:
# create file path
CWD = os.getcwd()
PTH = f'{CWD}/Data Engineer Scraping test - 17-06-21'

In [3]:
# get html and tree
html_page_link = 'candidateEvalData/webpage.html'
tree = html.parse(f'{PTH}/{html_page_link}')

In [6]:
# parse artist name
name = ' '.join(
    re.findall(r'[A-Z]\w+', tree.xpath('//h1[@id="main_center_0_lblLotPrimaryTitle"]/text()')[0])
    )
print(name)

Peter Doig


In [7]:
# parse painting name
painting = tree.xpath('//h2[@id="main_center_0_lblLotSecondaryTitle"]/i/text()')[0].strip()
print(painting)

The Architect's Home in the Ravine


In [8]:
# parse price GBP
price_gbp = ' '.join(
    re.findall(r'[0-9]+', tree.xpath('//span[@id="main_center_0_lblPriceRealizedPrimary"]/text()')[0])
    )
print(price_gbp)

11 282 500


In [18]:
# parse price USD (6 370 908 requested different from actual figure 16 370 908)
price_usd_req = ''.join(
    [p.replace(',', ' ') for p in
     re.findall(r'(\d{2,3},\d{2,3},\d{2,3})',
                tree.xpath('//div[@id="main_center_0_lblPriceRealizedSecondary"]/text()')[0])
    ])[1:]
# actual
price_usd_act = ''.join(
    [p.replace(',', ' ') for p in
     re.findall(r'(\d{2,3},\d{2,3},\d{2,3})',
                tree.xpath('//div[@id="main_center_0_lblPriceRealizedSecondary"]/text()')[0])
    ])
print(price_usd_req)
print(price_usd_act)

6 370 908
16 370 908


In [19]:
# parse price GBP est
est_gbp = (', ').join(
    [p.replace(',', ' ') for p in
     re.findall(r'(\d{2,3},\d{2,3},\d{2,3})',
                tree.xpath('//span[@id="main_center_0_lblPriceEstimatedPrimary"]/text()')[0])]
    )
print(est_gbp)

10 000 000, 15 000 000


In [28]:
# parse price USD est
est_usd = (', ').join(
    [p.replace(',', ' ') for p in
     re.findall(r'(\d{2,3},\d{2,3},\d{2,3})',
                tree.xpath('//span[@id="main_center_0_lblPriceEstimatedSecondary"]/text()')[0])]
    )
print(est_usd)

14 509 999, 21 764 999


In [21]:
# parse image link
image_url = tree.xpath('//img[@id="imgLotImage"]/@src')[0]
print(image_url)

http://www.christies.com/lotfinderimages/D59730/peter_doig_the_architects_home_in_the_ravine_d5973059g.jpg


In [23]:
# parse sale date
sale_date = datetime.strptime(
    re.findall(r'(\d{2}\s[A-Z]\w+\s\d{4})',
               tree.xpath('//span[@id="main_center_0_lblSaleDate"]/text()')[0])[0],'%d %B %Y'
    ).date()

sale_date = datetime.strftime(sale_date, '%Y-%m-%d')
print(sale_date)

2016-02-11


In [29]:
data = [{'Name of artist': name, 'Name of Painting': painting, 'Price realised GBP': price_gbp,
        'Price realised USD': price_usd_act, 'Estimates GBP': est_gbp, 'Estimates USD': est_usd,
        'Image url': image_url, 'Sale date': sale_date}]
df = pd.DataFrame(data)
df.head()

,Name of artist,Name of Painting,Price realised GBP,Price realised USD,Estimates GBP,Estimates USD,Image url,Sale date
0,Peter Doig,The Architect's Home in the Ravine,11 282 500,16 370 908,"10 000 000, 15 000 000","14 509 999, 21 764 999",http://www.christies.com/lotfinderimages/D5973...,2016-02-11
